In [ ]:
import os
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

: 

In [2]:
class_names = ['Safe', 'Medium', 'Dangerous', 'Super Dangerous']

In [9]:
df = pd.read_pickle('data_mapping.pkl')

df2 = df.iloc[:-1, :]
X, y = df2.iloc[:, 0:4], df2.iloc[:, 5]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5) # 20% is testing data



In [5]:
single_feature_normalizer = tf.keras.layers.Normalization(input_shape=[4,], axis=None)

model = tf.keras.Sequential([
  tf.keras.layers.Input(shape=(4,)),
  tf.keras.layers.Dense(10, activation=tf.nn.relu),  # input shape required
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(4)
])

In [6]:
# compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [1]:
model2 = DecisionTreeClassifier()
model2.fit(X_train, y_train)
pred = model2.predict(X_test)
accuracy_score(y_test, pred)

NameError: name 'DecisionTreeClassifier' is not defined

In [21]:
model.fit(features, labels, epochs=10)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'binary_crossentropy/Cast' defined at (most recent call last):
    File "/Users/brianchen/miniconda3/envs/new_env/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/brianchen/miniconda3/envs/new_env/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/brianchen/miniconda3/envs/new_env/lib/python3.8/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/Users/brianchen/miniconda3/envs/new_env/lib/python3.8/site-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/Users/brianchen/miniconda3/envs/new_env/lib/python3.8/site-packages/ipykernel/kernelapp.py", line 619, in start
      self.io_loop.start()
    File "/Users/brianchen/miniconda3/envs/new_env/lib/python3.8/site-packages/tornado/platform/asyncio.py", line 199, in st

In [22]:
predictions = model(features[:10])
predictions[:4]

tf.nn.softmax(predictions[:4])

# MAKE SURE EVERYTHING IS NUMBER

AttributeError: Exception encountered when calling layer "sequential" (type Sequential).

'tuple' object has no attribute 'rank'

Call arguments received by layer "sequential" (type Sequential):
  • inputs=       county_code  month  day  hour
21168           10      7    1     0
21169           10      7    1     1
21170           10      7    1     2
21171           10      7    1     3
21172           10      7    1     4
21173           10      7    1     5
21174           10      7    1     6
21175           10      7    1     7
21176           10      7    1     8
21177           10      7    1     9
  • training=None
  • mask=None

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [ ]:
def loss(model, x, y, training):
  # training=training is needed only if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  y_ = model(x, training=training)

  return loss_object(y_true=y, y_pred=y_)

l = loss(model, features, labels, training=False)
print("Loss test: {}".format(l))

Loss test: 1.072019338607788


In [ ]:
# change the weights based on the loss. 
def grad(model, inputs, targets):
  with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets, training=True)
  return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

In [ ]:
loss_value, grads = grad(model, features, labels)

print("Step: {}, Initial Loss: {}".format(optimizer.iterations.numpy(),
                                          loss_value.numpy()))

optimizer.apply_gradients(zip(grads, model.trainable_variables))

print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),
                                          loss(model, features, labels, training=True).numpy()))

Step: 0, Initial Loss: 1.072019338607788
Step: 1,         Loss: 1.0699684619903564


In [ ]:
## Note: Rerunning this cell uses the same model parameters

# Keep results for plotting
train_loss_results = []
train_accuracy_results = []

num_epochs = 201

for epoch in range(num_epochs):
  epoch_loss_avg = tf.keras.metrics.Mean()
  epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  # Training loop - using batches of 32
  for x, y in ds_train_batch:
    # Optimize the model
    loss_value, grads = grad(model, x, y)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # Track progress
    epoch_loss_avg.update_state(loss_value)  # Add current batch loss
    # Compare predicted label to actual label
    # training=True is needed only if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    epoch_accuracy.update_state(y, model(x, training=True))

  # End epoch
  train_loss_results.append(epoch_loss_avg.result())
  train_accuracy_results.append(epoch_accuracy.result())

  if epoch % 50 == 0:
    print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))

Epoch 000: Loss: 1.062, Accuracy: 35.581%
Epoch 050: Loss: 0.623, Accuracy: 79.775%
Epoch 100: Loss: 0.437, Accuracy: 86.517%
Epoch 150: Loss: 0.319, Accuracy: 96.255%
Epoch 200: Loss: 0.231, Accuracy: 97.004%
